# Cenário II

## Definindo o problema:

O segundo entregável consiste na transformação de dados disponíveis em <a href="https://drive.google.com/file/d/1IDCjpDZh5St97jw4K_bAewJ8hf-rax9C/view?usp=sharing">arquivo Json</a> para o formato de dataframe, algo comum no dia a dia da empresa. Após transformar esse Json em dataframe é possível perceber que a coluna "item_list" está como dicionário. Seu gestor pediu dois pontos de atenção nessa tarefa:

- Expandir a coluna num mesmo dataframe;
- Normalizar os itens dessa coluna de dicionário e dividí-los em dois dataframes separados, seguindo o modelo relacional.

In [ ]:
import json
import pandas as pd
import os
import gdown
import zipfile

In [ ]:
# faz o download do arquivo 'data.json'
output = 'data_cen1.zip'
url='https://drive.google.com/uc?id=1ggtgRwEsyvkUPerJYrRAu5E76y_8Ia6h&export=download'

gdown.download(url, output, quiet=False)

with zipfile.ZipFile(output,"r") as zip_ref:
      zip_ref.extractall()

os.remove(output)

Downloading...
From: https://drive.google.com/uc?id=1ggtgRwEsyvkUPerJYrRAu5E76y_8Ia6h&export=download
To: /content/data_cen1.zip
100%|██████████| 418/418 [00:00<00:00, 1.13MB/s]


In [ ]:
# abre o arquivo 'data.json'
f = open('data.json')

# load JSON file as object
data = json.load(f)

In [ ]:
# cria um novo dataframe 'df_Items' contendo as colunas de 'ItemList' bem como a chave primária 'NFeID'
df_Items = pd.DataFrame()

for i in data:
    NFeID=i['NFeID']
    ItemList=i['ItemList']
    df_=pd.json_normalize(ItemList)
    df_['NFeID']=NFeID
    df_Items=pd.concat([df_Items, df_])

In [ ]:
df_Items

,ProductName,Value,Quantity,NFeID
0,Rice,35.55,2,1
1,Flour,11.55,5,1
2,Bean,27.15,7,1
0,Tomate,12.25,10,2
1,Pasta,7.55,5,2
0,Beer,9.00,6,3
1,French fries,10.99,2,3
2,Ice cream,27.15,1,3


In [ ]:
#converte o json data.json em um dataframe pandas e exclui a coluna 'ItemList' pois não será mais necessária
df=pd.DataFrame.from_dict(data)
df.drop(columns=['ItemList'], inplace=True)

In [ ]:
df

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501,1
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,502,2
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,503,3


# EXPLICANDO O DESENVOLVIMENTO DA SOLUÇÃO:

A idéia para resolução desse cenário é aplicar a primeira forma normal, que diz que todos os atributos de uma tabela devem ser atômicos, ou seja, a tabela não deve conter atributos com mais de um valor como é o caso da coluna 'ItemList'. Para deixar nesta forma normal, é preciso identificar a chave primária da tabela, nesse caso 'NFeID', identificar a(s) coluna(s) que tem(êm) dados repetidos e removê-la(s), depois criar uma nova tabela com a chave primária para armazenar o dado repetido e, por fim, criar uma relação entre a tabela principal e a tabela secundária.
Dessa forma um novo dataframe foi criado chamado df_Items contendo os dados da coluna 'ItemList', incluindo a chave 'NFeID' do dataset original, então excluí a coluna 'ItemList' do dataframe df original criado a partir do json, pois não será mais necessária.
Agora temos 2 dataframes df e df_Items, ambos contendo a coluna 'NFeID' como chave, de acordo com o modelo entidade relacionamento de um banco relacional, e podemos estabelecer um relacionamento entre as 2 tabelas da seguinte forma:

In [ ]:
#exemplo de inner join
df.merge(df_Items, how='inner', on='NFeID')

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ProductName,Value,Quantity
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501,1,Rice,35.55,2
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501,1,Flour,11.55,5
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,501,1,Bean,27.15,7
3,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,502,2,Tomate,12.25,10
4,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,502,2,Pasta,7.55,5
5,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,503,3,Beer,9.00,6
6,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,503,3,French fries,10.99,2
7,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0.0,503,3,Ice cream,27.15,1


# PROPOSTA DE MELHORIA:

Acredito que seja possível utilizar estruturas de dados mais eficientes, sem que seja necessário separar as tabelas para atender a normalização, existe uma estrutura de dados do pacote xarray, que é similar ao dataframe bidimensional do pandas, mas permite a criação de estruturas com mais de duas dimensões, dessa forma poderia ser criada uma estrutura de dados multidimensional para acomodar os elementos da coluna 'ItemList'.